In [1]:
#-*- coding:utf-8 -*-

import os
import time
import datetime
import random
import math
import pickle
import unicodedata

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as st
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [2]:
cmap = matplotlib.cm.get_cmap('tab10')

In [3]:
order = ['raw']\
        + ['min-max_p_{}'.format(p) for p in [10,20,30,40]]+['min-max']\
        + ['z-score_p_{}'.format(p) for p in [10,20,30,40]]+['z-score']\
        + ['nonlinear_p_{}'.format(p) for p in [10,20,30,40]]+['nonlinear']
order

['raw',
 'min-max_p_10',
 'min-max_p_20',
 'min-max_p_30',
 'min-max_p_40',
 'min-max',
 'z-score_p_10',
 'z-score_p_20',
 'z-score_p_30',
 'z-score_p_40',
 'z-score',
 'nonlinear_p_10',
 'nonlinear_p_20',
 'nonlinear_p_30',
 'nonlinear_p_40',
 'nonlinear']

In [4]:
out_all = []
for lag in [5, 10, 15]:
    for var in [1, 2, 3]:
        f_name = 'results_{}_{}.csv'.format(lag, var)
        out_list = []
        for period in [10, 20, 30, 40]:
            out = pd.read_csv(os.path.join('results/out_{}'.format(period), f_name), index_col=0)
            out = out.rename({'min-max_p': 'min-max_p_{}'.format(period),
                              'z-score_p': 'z-score_p_{}'.format(period),
                              'nonlinear_p': 'nonlinear_p_{}'.format(period)})
            out_list.append(out)
        out_df = pd.concat(out_list).groupby(level=0).mean()
        out_df['lag'] = lag
        out_df['var'] = var
        out_df.set_index(['lag', 'var', order])
        out_all.append(out_df)

In [5]:
out_df.loc[order]

,mu(mean),std(mean),MAE(mean),mu(std),std(std),MAE(std),lag,var
raw,14.954550,9.490621,2.029850,0.221986,1.935205,0.238714,15,3
min-max_p_10,14.666400,47.858756,5.676800,0.244039,3.652747,0.267450,15,3
min-max_p_20,12.722600,10.846654,3.333400,0.267946,1.996543,0.072543,15,3
min-max_p_30,11.849400,14.134498,3.921000,0.235418,0.489095,0.117733,15,3
min-max_p_40,17.067600,14.294036,3.783600,0.058260,0.814836,0.162527,15,3
min-max,14.939800,9.405767,2.019600,0.210588,1.991697,0.240260,15,3
z-score_p_10,13.035600,41.041360,5.313200,0.194352,2.214085,0.177093,15,3
z-score_p_20,12.708400,27.357176,4.472800,0.300322,0.816626,0.132560,15,3
z-score_p_30,13.958600,21.795050,3.859000,0.347327,3.222836,0.281078,15,3
z-score_p_40,12.420000,20.695368,4.114000,0.237975,1.910790,0.085206,15,3


In [6]:
out_all_df = pd.concat(out_all)

In [7]:
out_all_df.to_csv('results/results.csv')
out_all_df

,mu(mean),std(mean),MAE(mean),mu(std),std(std),MAE(std),lag,var
min-max,7.0411,38.491792,3.3252,0.254727,2.926045,0.192717,5,1
min-max_p_10,14.8556,33.032212,10.1372,0.412961,1.192325,0.381979,5,1
min-max_p_20,12.2532,66.790092,7.8160,0.218627,3.780614,0.198404,5,1
min-max_p_30,5.5380,4.962852,1.7564,0.591696,1.868927,0.484272,5,1
min-max_p_40,6.4214,10.228630,2.0714,0.091608,0.533675,0.053442,5,1
...,...,...,...,...,...,...,...,...
z-score,14.9497,9.439623,2.0230,0.210564,2.096249,0.249075,15,3
z-score_p_10,13.0356,41.041360,5.3132,0.194352,2.214085,0.177093,15,3
z-score_p_20,12.7084,27.357176,4.4728,0.300322,0.816626,0.132560,15,3
z-score_p_30,13.9586,21.795050,3.8590,0.347327,3.222836,0.281078,15,3


In [8]:
out_all_df = out_all_df.reset_index()
out_all_df = out_all_df.rename({'index': 'model'}, axis=1)

In [9]:
results = pd.pivot_table(out_all_df, values='mu(mean)', index='model', columns=['lag', 'var']).loc[order]
results.to_csv('results/results_mu.csv')
results

lag                    5                              10                      \
var                     1         2          3         1        2          3   
model                                                                          
raw              7.052050  10.38315   5.781200   9.13025  13.3655  11.882200   
min-max_p_10    14.855600  11.94040  16.917600  19.66280  14.6604  15.017800   
min-max_p_20    12.253200  13.84380   6.500400  13.85240  14.8748  10.518600   
min-max_p_30     5.538000   9.77780   4.138600   9.30620  10.6886   8.883600   
min-max_p_40     6.421400   6.63780   6.261200  13.00980  13.0546  10.536600   
min-max          7.041100  10.39830   5.768100   9.13445  13.3689  11.880850   
z-score_p_10     8.129200   9.72720  10.699400  11.84940  12.8290  10.971000   
z-score_p_20     7.963200  10.79000   7.567400  12.26640  13.1162  10.747000   
z-score_p_30     8.149200  12.17140   6.879200  11.53440  14.0974  10.466200   
z-score_p_40     6.659800  12.14280   7.321000  12.45300  11.9150  10.180800   
z-score          7.040850  10.37815   5.777850   9.13615  13.3720  11.881100   
nonlinear_p_10   8.036000  13.07940  14.941400  10.63060  14.6312  11.056400   
nonlinear_p_20   5.022200   7.96060   6.753800  10.69440  13.6698  10.961200   
nonlinear_p_30   5.683800   6.13960   6.069800  10.72280  12.7870  11.094800   
nonlinear_p_40   5.272800   5.93760   4.346600   9.51280   9.5018   9.849800   
nonlinear        5.786267   9.06420   5.409867   9.15000  13.3182  11.779333   

lag                    15                        
var                     1          2          3  
model                                            
raw             13.690500  14.408350  14.954550  
min-max_p_10    16.626000  17.024400  14.666400  
min-max_p_20    14.279800  15.460400  12.722600  
min-max_p_30    13.574200  15.073400  11.849400  
min-max_p_40    16.560200  16.640400  17.067600  
min-max         13.691500  14.400450  14.939800  
z-score_p_10    14.550200  14.555000  13.035600  
z-score_p_20    14.266800  14.558800  12.708400  
z-score_p_30    15.212800  17.461000  13.958600  
z-score_p_40    14.940000  16.159800  12.420000  
z-score         13.695000  14.409400  14.949700  
nonlinear_p_10  15.044200  12.442200  14.246200  
nonlinear_p_20  14.684000  15.787400  13.309800  
nonlinear_p_30  15.751200  16.241600  17.604000  
nonlinear_p_40  13.460200  13.364000  11.046800  
nonlinear       14.454667  14.108067  14.361667

In [10]:
results = pd.pivot_table(out_all_df, values='std(mean)', index='model', columns=['lag', 'var']).loc[order]
results.to_csv('results/results_std.csv')
results

lag                    5                                10             \
var                     1          2          3          1          2   
model                                                                   
raw             38.629152  52.715392   6.952660   6.219948  40.735003   
min-max_p_10    33.032212  27.054012  39.088468  25.008444  37.106032   
min-max_p_20    66.790092  39.799762  28.202612  37.183260  24.756244   
min-max_p_30     4.962852  56.496686   7.030702   4.330290  15.744914   
min-max_p_40    10.228630  15.200518   7.159684  17.588398   9.072298   
min-max         38.491792  52.714197   6.902539   6.236303  40.726609   
z-score_p_10    42.672384  53.186020  46.170794  23.686606  38.396338   
z-score_p_20    30.773528  55.878280  26.031974  23.753764  38.576222   
z-score_p_30    29.559648  66.750174  21.159232  12.431188  12.230646   
z-score_p_40     7.435166  63.993916  22.085550  21.441094  15.009718   
z-score         38.481647  52.609731   6.958013   6.236590  40.785188   
nonlinear_p_10  39.186424  50.863970  48.709150  12.150182  52.290108   
nonlinear_p_20   1.817842  23.863774  34.330166   8.555976  11.534526   
nonlinear_p_30   9.135638   6.176200   5.131946   8.190532   7.443626   
nonlinear_p_40   3.071348   3.399524   5.242890   3.152532   3.635534   
nonlinear       25.147675  47.375438   8.064436   7.336957  42.600742   

lag                               15                        
var                     3          1          2          3  
model                                                       
raw             11.567000   8.320958   4.623990   9.490621  
min-max_p_10    43.497758  18.871056  39.302184  47.858756  
min-max_p_20    13.190982   9.891462  24.718132  10.846654  
min-max_p_30    11.217548  10.172610  12.913686  14.134498  
min-max_p_40     3.212006  15.097578  17.333568  14.294036  
min-max         11.540020   8.352984   4.588691   9.405767  
z-score_p_10    37.027338  24.208454  43.069738  41.041360  
z-score_p_20    31.415974  35.654628  27.729828  27.357176  
z-score_p_30    11.465970   9.862280  16.855606  21.795050  
z-score_p_40    10.168808  21.089752  32.854842  20.695368  
z-score         11.537369   8.327958   4.614999   9.439623  
nonlinear_p_10  11.445648  10.565850  38.394002  37.207894  
nonlinear_p_20   3.475012   5.770840   8.467906  16.961846  
nonlinear_p_30   2.640584   6.641632   8.023560  11.079352  
nonlinear_p_40  10.439738   8.243642   5.895164  12.536196  
nonlinear       10.432456   5.461695   5.777995   7.626031

In [11]:
results = pd.pivot_table(out_all_df, values='MAE(mean)', index='model', columns=['lag', 'var']).loc[order]
results.to_csv('results/results_MAE.csv')
results

lag                   5                              10                   \
var                    1        2          3          1       2        3   
model                                                                      
raw              3.33655  6.19155   1.541700   1.494050  4.8514  2.70270   
min-max_p_10    10.13720  7.50840  12.068400  10.050800  6.4896  6.78940   
min-max_p_20     7.81600  9.18260   3.639600   4.994400  5.8080  2.81340   
min-max_p_30     1.75640  5.84860   2.349000   1.659400  2.9702  2.63080   
min-max_p_40     2.07140  2.71700   2.302000   3.772200  3.3150  1.42820   
min-max          3.32520  6.20240   1.535600   1.494450  4.8493  2.69935   
z-score_p_10     4.21640  6.12040   6.717400   3.353800  5.3382  5.10860   
z-score_p_20     3.38560  6.97640   3.999400   3.572000  5.7998  4.35340   
z-score_p_30     3.68680  8.05140   3.679200   2.228000  4.5578  2.43660   
z-score_p_40     1.78660  7.67920   3.781000   3.364600  3.5022  2.36720   
z-score          3.32675  6.18115   1.540550   1.495650  4.8575  2.69960   
nonlinear_p_10   3.49120  8.69180  10.027000   2.085800  7.2884  1.61400   
nonlinear_p_20   0.69260  3.52140   3.528600   1.690400  4.0026  1.11600   
nonlinear_p_30   1.45940  1.81960   1.778600   1.728400  2.9774  1.49720   
nonlinear_p_40   1.37320  1.73240   2.023400   1.292400  1.4642  2.47460   
nonlinear        2.35440  5.04580   1.649733   1.796533  4.9978  2.60560   

lag                 15                      
var                  1         2         3  
model                                       
raw             1.8247  1.480350  2.029850  
min-max_p_10    3.4316  5.863200  5.676800  
min-max_p_20    1.9618  3.964800  3.333400  
min-max_p_30    2.3442  2.660200  3.921000  
min-max_p_40    3.0770  3.386000  3.783600  
min-max         1.8287  1.476750  2.019600  
z-score_p_10    3.0486  5.539000  5.313200  
z-score_p_20    4.1592  4.036400  4.472800  
z-score_p_30    1.5696  3.933800  3.859000  
z-score_p_40    2.7104  4.785000  4.114000  
z-score         1.8240  1.478500  2.023000  
nonlinear_p_10  2.0274  5.519400  4.318200  
nonlinear_p_20  1.4180  2.018600  3.335000  
nonlinear_p_30  1.6080  2.330000  3.464400  
nonlinear_p_40  2.3162  2.186000  4.228800  
nonlinear       1.2980  1.805667  1.924733

In [12]:
out_all_df['Err'] = abs(out_all_df['mu(mean)']-out_all_df['lag'])

In [13]:
results = pd.pivot_table(out_all_df, values='Err', index='model', columns=['lag', 'var']).loc[order]
results.to_csv('results/results_Err.csv')
results

lag                   5                            10                    \
var                    1        2          3        1       2         3   
model                                                                     
raw             2.052050  5.38315   0.781200  0.86975  3.3655  1.882200   
min-max_p_10    9.855600  6.94040  11.917600  9.66280  4.6604  5.017800   
min-max_p_20    7.253200  8.84380   1.500400  3.85240  4.8748  0.518600   
min-max_p_30    0.538000  4.77780   0.861400  0.69380  0.6886  1.116400   
min-max_p_40    1.421400  1.63780   1.261200  3.00980  3.0546  0.536600   
min-max         2.041100  5.39830   0.768100  0.86555  3.3689  1.880850   
z-score_p_10    3.129200  4.72720   5.699400  1.84940  2.8290  0.971000   
z-score_p_20    2.963200  5.79000   2.567400  2.26640  3.1162  0.747000   
z-score_p_30    3.149200  7.17140   1.879200  1.53440  4.0974  0.466200   
z-score_p_40    1.659800  7.14280   2.321000  2.45300  1.9150  0.180800   
z-score         2.040850  5.37815   0.777850  0.86385  3.3720  1.881100   
nonlinear_p_10  3.036000  8.07940   9.941400  0.63060  4.6312  1.056400   
nonlinear_p_20  0.022200  2.96060   1.753800  0.69440  3.6698  0.961200   
nonlinear_p_30  0.683800  1.13960   1.069800  0.72280  2.7870  1.094800   
nonlinear_p_40  0.272800  0.93760   0.653400  0.48720  0.4982  0.150200   
nonlinear       0.786267  4.06420   0.409867  0.85000  3.3182  1.779333   

lag                   15                      
var                    1         2         3  
model                                         
raw             1.309500  0.591650  0.045450  
min-max_p_10    1.626000  2.024400  0.333600  
min-max_p_20    0.720200  0.460400  2.277400  
min-max_p_30    1.425800  0.073400  3.150600  
min-max_p_40    1.560200  1.640400  2.067600  
min-max         1.308500  0.599550  0.060200  
z-score_p_10    0.449800  0.445000  1.964400  
z-score_p_20    0.733200  0.441200  2.291600  
z-score_p_30    0.212800  2.461000  1.041400  
z-score_p_40    0.060000  1.159800  2.580000  
z-score         1.305000  0.590600  0.050300  
nonlinear_p_10  0.044200  2.557800  0.753800  
nonlinear_p_20  0.316000  0.787400  1.690200  
nonlinear_p_30  0.751200  1.241600  2.604000  
nonlinear_p_40  1.539800  1.636000  3.953200  
nonlinear       0.545333  0.891933  0.638333